In [1]:
import argparse,numpy as np,  os, pickle, yaml, torch
from src.TorchSimulation.transmitter import simpleWDMTx,choose_sps
from src.TorchSimulation.channel import manakov_ssf, choose_dz, get_beta2
from src.TorchSimulation.receiver import simpleRx, BER
import warnings
warnings.filterwarnings('error')

with open('configs/simulation/test.yaml', 'r') as file:
    args = yaml.safe_load(file)

# Set Global params
M = args['M']                   # QAM format.
Ltotal = args['Ltotal']         # Total Length. [km]
Lspan = args['Lspan']           # Length per span. [km]
alpha = args['alpha']           # Attenuation parameter. [dB/km]
D = args['D']                   # Dispersion parameter.  [ps/nm/km]
Dpmd = args['Dpmd']             # PMD parameter. [ps/sqrt(km)]
PMD = args['PMD']               # Simulation PMD or not. True or False.
Lcorr = args['Lcorr']           # Fiber Correlation length. [km]
Fc = args['Fc']                 # Central frequency [Hz]
gamma = args['gamma']           # Nonlinear parameter. [/W/km]
amp = args['amp']               # Amplifier type. 'edfa', 'ideal', or None. 
NF = args['NF']                 # EDFA Noise Figure. [dB]
beta2 = get_beta2(D, Fc)        # [s^2/km]
spacing_factor = args['SF'] # freqspace/Rs

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for Nch in args['Nch']:
    for Rs in args['Rs']:
        Rs = Rs * 1e9   # symbol rate [Hz]
        for Pch_dBm in args['Pch']:
            print(f'\n#######      Nch:{Nch}       Rs:{int(Rs/1e9)}GHz     Pch:{Pch_dBm}dBm        #######', flush=True)
            freqspace = Rs * spacing_factor
            sps = choose_sps(Nch, freqspace, Rs)
            hz = choose_dz(freqspace, Lspan, Pch_dBm, Nch, beta2, gamma)
            print(f'#######     Tx sps = {sps},  simulation hz={hz}km       #######', flush=True)
            
            tx_seed = np.random.randint(0, 2**32)
            ch_seed = np.random.randint(0, 2**32)
            rx_seed = np.random.randint(0, 2**32)

            tx_path = args['path'] + f'/Tx_Nch{Nch}_{int(Rs/1e9)}GHz_Pch{Pch_dBm}dBm'
            channel_path = args['path'] + f'/Channel_Nch{Nch}_{int(Rs/1e9)}GHz_Pch{Pch_dBm}dBm'

            ## Step 1: Tx
            tx_data = simpleWDMTx(tx_seed, args['batch'], M, args['Nbits'], sps, Nch, args['Nmodes'], Rs, freqspace, Pch_dBm, device=device)
            with open(tx_path, 'wb') as file: pickle.dump(tx_data['config'], file)

            ## Step 2: channel
            trans_data = manakov_ssf(tx_data, ch_seed, Ltotal, Lspan, hz, alpha, D, gamma, Fc,amp, NF,order=1, openPMD=PMD, device=device)
            with open(channel_path, 'wb') as file: pickle.dump(trans_data, file)

            # ## Step 3: Rx 
            # rx_data = simpleRx(rx_seed, trans_data, tx_data['config'], Nch//2, rx_sps=2, method='frequency cut')


#######      Nch:3       Rs:40GHz     Pch:-1dBm        #######
#######     Tx sps = 4,  simulation hz=0.5km       #######


Sample rate fa: 1.6e+11, Cut off frequency fc: 7.2e+10, fa > 2fc: True
 simpleWDMTx complete, time cost(s):0.722419261932373
 manakov_ssf complete, time cost(s):98.14984631538391


In [3]:
rx_data = simpleRx(rx_seed, trans_data['signal'], tx_data['config'], Nch//2, rx_sps=2, method='frequency cut', device=device)

In [ ]:
# {
#     'Tx':  tx_data['SymbTx'][:,:,2,:],
#     'Rx(sps=2)':  rx_data['signal'],
#     'Rx(sps=1)':  
# }

In [4]:
from src.TorchDSP.baselines import _CDCDSP, CDC, DDLMS
Rx = rx_data['signal']
Tx = tx_data['SymbTx'][:,:,tx_data['config']['Nch']//2,:].to(torch.complex64)/np.sqrt(10)
Fs = torch.tensor([2*tx_data['config']['Rs']]*args['batch'])
print(Rx.shape, Tx.shape, Fs.shape)

torch.Size([20, 400000, 2]) torch.Size([20, 200000, 2]) torch.Size([20])


In [5]:
E = CDC(Rx.to('cuda:0'), Fs.to('cuda:0'),  2000e3)  # [batch, Nfft, Nmodes]

In [6]:
F = DDLMS(E.to('cpu'), Tx.to('cpu'), sps=2, lead_symbols=2000)

In [7]:
F = DDLMS(E.to('cuda:0'), Tx.to('cuda:0'), sps=2, lead_symbols=2000)

In [23]:
np.mean(BER(F.val[0,10000:], Tx[:,F.t.start:F.t.stop,:][0,10000:])['Qsq'])

8.507282

In [24]:
import numpy as np, matplotlib.pyplot as plt, pickle
from src.TorchDSP.dataloader import get_k, get_k_batch
res, code = pickle.load(open('_outputs/Qfactor/Nmodes2/CDC.pkl','rb'))
P = np.arange(-8, 9)
Rs = 40


k = get_k_batch(3, Rs, code)
Q = np.mean(res['BER from 10000th symb']['CDC']['Qsq'][k], axis=-1)

Q[7]
# Q = np.mean(Q, axis=1)
# plt.plot(P, Q)
# plt.ylim(6, 10)

Exception ignored in: <_io.FileIO name='_outputs/Qfactor/Nmodes2/CDC.pkl' mode='rb' closefd=True>
Traceback (most recent call last):
  File "/tmp/ipykernel_137271/2471261490.py", line 3, in <module>


8.5072365